In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quadrature
from scipy.optimize import least_squares 
from datetime import datetime as dt
from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
import py_vollib_vectorized

In [3]:
S0=15922

In [4]:
df1= pd.read_csv('/content/dyh23-volatility-greeks-exp-03_18_23-show-all-02-04-2023.csv')
df2= pd.read_csv('/content/dym23-volatility-greeks-exp-04_22_23-%moneyness%-02-04-2023.csv')
df3= pd.read_csv('/content/dym23-volatility-greeks-exp-06_17_23-%moneyness%-02-04-2023.csv')
df4= pd.read_csv('/content/dyu23-volatility-greeks-exp-09_16_23-%moneyness%-02-04-2023.csv')
df5= pd.read_csv('/content/dyz23-volatility-greeks-exp-12_16_23-%moneyness%-02-04-2023.csv')

In [5]:
df1['exp']=dt(2023, 3, 18)
df2['exp']=dt(2023, 4, 22)
df3['exp']=dt(2023, 6, 17)
df4['exp']=dt(2023, 9, 23)
df5['exp']=dt(2023, 12, 23)

In [6]:
df_call = pd.concat([df1[df1['Type']=='Call'], df2[df2['Type']=='Call'], df3[df3['Type']=='Call'], df4[df4['Type']=='Call'], df5[df5['Type']=='Call']], ignore_index=True)
df_put = pd.concat([df1[df1['Type']=='Put'], df2[df2['Type']=='Put'], df3[df3['Type']=='Put'], df4[df4['Type']=='Put'], df5[df5['Type']=='Put']], ignore_index=True)

In [7]:
df_call['Time']=dt(2023, 2, 4)
df_put['Time']=dt(2023, 2, 4)

In [8]:
df_call['tau']=(df_call['exp']-df_call['Time']).dt.days/365
df_put['tau']=(df_put['exp']-df_put['Time']).dt.days/365

In [9]:
df_call['IV']=df_call['IV'].str.strip('%').to_numpy('float')
df_put['IV']=df_put['IV'].str.strip('%').to_numpy('float')

In [ ]:
df_call.head()

,Strike,Type,Last,IV,Delta,Gamma,Theta,Vega,IV Skew,Time,exp,Last Trade,tau
0,4500,Call,10979.5,172.50,0.994668,4.243346e-115,-4.015857e-112,1.896728e-110,+158.26%,2023-02-04,2023-03-18,NaN,0.115068
1,5000,Call,10480.9,157.50,0.994668,5.154146e-97,-5.884828e-94,2.303845e-92,+143.26%,2023-02-04,2023-03-18,NaN,0.115068
2,5500,Call,9982.2,143.94,0.994668,4.872158e-82,-6.592205e-79,2.177800e-77,+129.69%,2023-02-04,2023-03-18,NaN,0.115068
3,6000,Call,9483.6,131.69,0.994668,1.627567e-69,-2.571346e-66,7.275042e-65,+117.45%,2023-02-04,2023-03-18,NaN,0.115068
4,6500,Call,8984.9,120.41,0.994668,5.819387e-59,-1.060279e-55,2.601200e-54,+106.16%,2023-02-04,2023-03-18,NaN,0.115068


In [10]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    
    # constants
    a = kappa*theta
    b = kappa+lambd
    
    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j
    
    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )
    
    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)
    
    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
    return exp1*term2*exp2

In [11]:
#v0, kappa, theta, sigma, rho, lambd
def integrand(phi, S0,  v0, kappa, theta, sigma, rho, lambd, tau, r, K):

  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  
  
  numerator = np.exp(r*tau) * heston_charfunc(phi-1j,*args) - K*heston_charfunc(phi,*args)
  
  denominator = 1j*phi*K**(1j*phi)
  return numerator/denominator

In [12]:
def vectorized_quad(integrand, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    result = np.zeros(len(K))
    for i in range(len(K)):
        t = tau[i]
        result[i], _ = quadrature(integrand, 0, 100, maxiter = 25, args=(S0, v0, kappa, theta, sigma, rho, lambd, t, r[i], K[i]))
    return result

In [24]:
def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):

  
  
  real_integral = np.real(vectorized_quad(integrand, S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r) )
    
  return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi + K*np.exp(-r*tau)-S0

Where I found the rates for the Germany Governments bonds
http://www.worldgovernmentbonds.com/country/germany/

In [14]:
rates = {1/12:	2.440,	
3/12:	2.5782,	
6/12:	2.725,	
9/12:	2.940,	
1:	2.919,	
2:	2.765,	
3:  2.516,	
4:	2.409,	
5:	2.399,	
6:	2.319,	
7:	2.303,	
8:	2.303,	
9:	2.313,	
10:	2.362,	
15:	2.439,	
20:	2.410,	
25:	2.328,	
30:	2.320}	

In [15]:
mat = []
rate = []
for i, j in rates.items():
  mat.append(i)
  rate.append(j/100)
mat = np.array(mat)
rate = np.array(rate)

In [16]:
curve_fit, status = calibrate_nss_ols(mat, rate) 
curve_fit

NelsonSiegelSvenssonCurve(beta0=0.027281114404616363, beta1=-0.0008170676412227167, beta2=0.014071560272008411, beta3=-0.023836160555950073, tau1=2.0, tau2=5.0)

In [17]:
df_call['r']= df_call['tau'].apply(curve_fit)
df_put['r']= df_put['tau'].apply(curve_fit)
 # initial asset price VALUES to test the function and possible itial values
K = df_put['Strike'].to_numpy('float') # strike

tau = df_put['tau']#df_call['tau'] # time to maturity
r = df_put['r']

In [ ]:
a = heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

<ipython-input-13-7b73f4590402>:5: ComplexWarning:

Casting complex values to real discards the imaginary part



In [ ]:
a.isna().sum()

0

In [ ]:
a[a<0]

Series([], dtype: float64)

In [19]:
iv=df_put['IV']/100

In [18]:
def heston_ivol(x):
  v0, kappa, theta, sigma, rho, lambd = [param for param in x]
  P = heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
  impl = py_vollib_vectorized.vectorized_implied_volatility(P, S0, K, tau, r, flag = ['p'], q=0, model='black_scholes',return_as='numpy', on_error='ignore')
  diff = iv - impl
  return diff


In [ ]:
b=heston_ivol([v0, kappa, theta, sigma, rho, lambd])

<ipython-input-13-7b73f4590402>:5: ComplexWarning:

Casting complex values to real discards the imaginary part



In [25]:
def calibrate_heston(df, S0, x, r):
    
  K=df['Strike']
  tau=df['tau']
  iv = df['IV']/100
  r=df['r']
  S0=S0

  #market_ivol = heston_ivol(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r, iv)
  res = least_squares(heston_ivol, x0=np.array(x), verbose=0, max_nfev=100, method='lm')
  return res.x

In [26]:
#v0, kappa, theta, sigma, rho, lambd
v0_dict = {'a': 0.1838, 'b': 0.0400, 'c':0.1500, 'd': 0.0231, 'e':0.0654}
kappa_dict = {'a': 6.5482, 'b': 1.1500, 'c': 3.0000, 'd': 1.3784, 'e': 0.6067}
theta_dict = {'a': 0.0731, 'b': 0.0400, 'c': 0.0500, 'd': 0.2319, 'e': 0.0707}
sigma_dict = {'a': 2.3012, 'b': 0.2000, 'c': 0.5000, 'd':1.0359, 'e': 0.2928}
rho_dict = {'a': -0.4176, 'b': -0.4000, 'c': -0.5000, 'd': -0.2051, 'e': -0.7571}
lambd_dict = {'a':0, 'b':0, 'c':0, 'd':0, 'e':0}


In [ ]:
params={}
for i in v0_dict.keys():
  
  params[i]=calibrate_heston(df_put, S0, [v0_dict[i], kappa_dict[i], theta_dict[i], sigma_dict[i], rho_dict[i], lambd_dict[i]], r)

In [28]:
params['a']

array([4.59415716e-01, 2.21761166e+01, 1.28448769e-03, 1.09321710e-01,
       1.37031920e+00, 1.56147632e+01])

In [29]:
params['b']

array([ 0.1072635 ,  1.15881954, -0.10824569,  0.14290759, -0.37484974,
        0.00789902])

In [30]:
params['c']

array([ 0.1205534 ,  3.02506064, -0.04752856,  0.12505951, -0.35316273,
        0.02944916])

In [31]:
params['d']

array([ 0.10689703,  1.41315015, -0.10502918,  0.07116143,  0.15383682,
        0.06715762])

In [32]:
params['e']

array([ 0.1614243 , 18.25808615,  0.10135218, -0.62784279, -1.7754437 ,
        7.46237681])

In [33]:
err = {}
for i in params.keys():
  err[i]=(heston_ivol(params[i].tolist())**2).sum()/K.shape[0]

<ipython-input-12-7b73f4590402>:5: ComplexWarning:

Casting complex values to real discards the imaginary part



In [34]:
err

{'a': 0.05017240253212596,
 'b': 0.051212116112885286,
 'c': 0.05114459095955435,
 'd': 0.052959684980672175,
 'e': 0.048344266632188895}

In [20]:
def calcImpl(x):
  v0, kappa, theta, sigma, rho, lambd = [param for param in x]
  P = heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
  impl = py_vollib_vectorized.vectorized_implied_volatility(P, S0, K, tau, r, flag = ['p'], q=0, model='black_scholes',return_as='numpy', on_error='ignore')
  return impl

In [ ]:
df_put['predIV']=calcImpl(params['a'])

In [39]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface

fig = go.Figure(data=[go.Mesh3d(x=df_put['tau'], y=df_put['Strike'], z=df_put['IV']/100, color='mediumblue', opacity=0.55)])
fig.add_scatter3d(x=df_put['tau'], y=df_put['Strike'], z=df_put['predIV'], mode='markers')
fig.update_layout(
    title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
    scene = dict(xaxis_title='TIME (Years)',
                    yaxis_title='STRIKES (Pts)',
                    zaxis_title='INDEX OPTION PRICE (Pts)'),
    height=800,
    width=800
)
fig.show()

In [40]:
def calibrate_heston(df, S0, x, r):
    
  K=df['Strike']
  tau=df['tau']
  iv = df['IV']/100
  r=df['r']
  S0=S0

  #market_ivol = heston_ivol(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r, iv)
  res = least_squares(heston_ivol, x0=np.array(x), verbose=0, max_nfev=100, method='trf',
                      bounds=([0, 1e-3, 1e-3, 1e-2, -1, -1], [np.inf, np.inf, np.inf, np.inf, 0, np.inf]))
  return res.x

In [ ]:
params_trf={}
for i in v0_dict.keys():
  try:
    params_trf[i]=calibrate_heston(df_put, S0, [v0_dict[i], kappa_dict[i], theta_dict[i], sigma_dict[i], rho_dict[i], lambd_dict[i]], r)
  except:
    params_trf[i]='not avaible'

In [ ]:
err_tfr = {}
for i in params_trf.keys():
  if params_trf[i]!='not avaible':

    err_tfr[i]=(heston_ivol(params_trf[i].tolist())**2).sum()/K.shape[0]

In [53]:
params_trf

{'a': array([ 3.11124119e-01,  2.20153976e+01,  5.06478378e-03,  5.48331459e-01,
        -1.91585687e-01,  5.95924568e+00]),
 'b': array([ 1.17087664e-01,  3.08266191e+00,  1.00000005e-03,  1.66510356e-01,
        -2.55128557e-01,  7.13704790e-01]),
 'c': array([ 1.39595877e-01,  5.80301341e+00,  1.00000499e-03,  9.05140915e-02,
        -4.09346798e-01,  1.44764346e+00]),
 'd': array([ 1.28229689e-01,  3.86690825e+00,  1.00028180e-03,  1.12044169e-01,
        -3.39758014e-01,  1.69229079e+00]),
 'e': array([ 1.15296066e-01,  1.72783523e+00,  1.00000375e-03,  1.08195110e-01,
        -3.39401631e-01,  1.78998004e+00])}

In [55]:
err_tfr

{'a': 0.04984925150303156,
 'b': 0.05119214511292639,
 'c': 0.05050913628939209,
 'd': 0.050745611428641355,
 'e': 0.0512739024991142}

In [56]:
df_put['predIV']=calcImpl(params_trf['a'])

<ipython-input-12-7b73f4590402>:5: ComplexWarning:

Casting complex values to real discards the imaginary part



In [57]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface

fig = go.Figure(data=[go.Mesh3d(x=df_put['tau'], y=df_put['Strike'], z=df_put['IV']/100, color='mediumblue', opacity=0.55)])
fig.add_scatter3d(x=df_put['tau'], y=df_put['Strike'], z=df_put['predIV'], mode='markers')
fig.update_layout(
    title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
    scene = dict(xaxis_title='TIME (Years)',
                    yaxis_title='STRIKES (Pts)',
                    zaxis_title='INDEX OPTION PRICE (Pts)'),
    height=800,
    width=800
)
fig.show()